In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
"""
Created on Mon Feb 18 20:30:48 2019

@author: jschmidt
"""

import os, shutil, random

import pandas as pd
import sys
import gdal
import imp
import numpy as np


import scripts.windturbines.functions_pattern_recognition as fpr
imp.reload(fpr)
from scripts.windturbines.functions_pattern_recognition import get_param
from scripts.windturbines.functions_pattern_recognition import cop_predict
from scripts.windturbines.functions_pattern_recognition import check_image
from scripts.windturbines.functions_pattern_recognition import read_params

Using TensorFlow backend.
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

In [3]:
COUNTRY = "US"

train_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TRAIN")
test_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_TEST")
validation_dir = get_param(COUNTRY,"PATH_ML_IMAGES_TYPE_REGRESSION_VALIDATION")


#### delete directories if exist
#### create if not exist

shutil.rmtree(train_dir,ignore_errors=True)
shutil.rmtree(test_dir,ignore_errors=True)
shutil.rmtree(validation_dir,ignore_errors=True)

os.makedirs(train_dir)
os.makedirs(test_dir)
os.makedirs(validation_dir)

src_dir_tb = get_param(COUNTRY,"PATH_RAW_IMAGES_TURBINES")


In [6]:
turbines_us = pd.read_csv(get_param("US","FILE_TURBINE_LOCATIONS"))

predictions_cs = get_param("US",
                            "PATH_RAW_IMAGES_TURBINES")

#turbines_pred = pd.read_feather(predictions_cs + "all_predictions.feather")

turbines_pred = pd.read_feather("data/aerialImages/GOOGLE/RESOLUTION19/US/raw/Turbines/all_predictions.feather")

turbines_us = turbines_us[turbines_pred['prediction'] > 0.99]

/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [7]:
turbines_us['id'] = np.arange(1, turbines_us.shape[0] + 1)

In [8]:
nmb_samples_all = turbines_us.shape[0]

turbines_us = turbines_us[turbines_us['id'] <= nmb_samples_all]
turbines_us.shape

(47665, 28)

In [9]:
share_train = nmb_samples_all * 0.8


In [10]:
cnt = 0
for i in range(0, nmb_samples_all):
    cnt += 1
        
        
    #print(file)
    file_s = str(turbines_us.iloc[[i]]['id'].values[0] + 1) + '.tif'
    file_d = str(turbines_us.iloc[[i]]['id'].values[0] + 1) + '.png'
        
    #print(file)
        
    src = os.path.join(src_dir_tb, file_s)
        
    if(cnt % 1000 == 0):
        print(cnt)
        print(src)
            
    #print(src)
        
        
        
    if(os.path.isfile(src)):
            
        dst = ""
        
        if(cnt < share_train):
           
            dst = os.path.join(train_dir, file_d)
            
        if(cnt > share_train):
           
            dst = os.path.join(test_dir, file_d)
            
        fpr.copyfile_to_png(src, dst)
            
    #### copy no-turbine images

1000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/1001.tif
2000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/2001.tif
3000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/3001.tif
4000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/4001.tif
5000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/5001.tif
6000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/6001.tif
7000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/7001.tif
8000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/8001.tif
9000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/9001.tif
10000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/10001.tif
11000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/11001.tif
12000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/12001.tif
13000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/13001.tif
14000
data/aerialImages/GOOGLE/RESOLUTION13/US/raw/Turbines/14001.tif
15000
data/aerialImages/GOOGLE/RESOLUT

In [11]:
print('total training turbine images ', len(os.listdir(train_dir)))
print('total testing turbine images ', len(os.listdir(test_dir)))
print('total validation turbine images ', len(os.listdir(validation_dir)))



pd_out = turbines_us['id'].map(str) + '.png'
pd_out['score'] = turbines_s['t_hh']

pd_out_train = pd_out.iloc[[range(0, len(os.listdir(train_dir)))]]
pd_out_test = pd_out.iloc[[range(len(os.listdir(train_dir) + 1, len(os.listdir(test_dir)))]
pd_out_validation = pd_out.iloc[[range(len(os.listdir(test_dir)), len(os.listdir(validation_dir))]]
                                      
pd_out_train.to_csv(train_dir + "list.csv")
pd_out_test.to_csv(test_dir + "list.csv")
pd_out_validation.to_csv(validation_dir + "list.csv")


SyntaxError: invalid syntax (<ipython-input-11-042fd762f347>, line 11)

In [12]:

print('total training turbine images ', len(os.listdir(train_dir)))
print('total testing turbine images ', len(os.listdir(test_dir)))

pd_out = turbines_us[['id','t_hh']]

pd_out['id_s'] = turbines_us['id'].map(str) + '.png'
pd_out['t_hh'] = turbines_us['t_hh']
pd_out['t_rd'] = turbines_us['t_rd']
pd_out['t_cap'] = turbines_us['t_cap']

share_train_i = len(os.listdir(train_dir))

pd_out_train = pd_out.iloc[0: share_train_i]
pd_out_test = pd_out.iloc[((share_train_i) + 1) : pd_out.shape[0]]
                                      
pd_out_train.to_csv(train_dir + "list.csv")
pd_out_test.to_csv(test_dir + "list.csv")
pd_out

total training turbine images  15046
total testing turbine images  4927


/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/jschmidt/.conda/envs/windturbine-identification/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,id,t_hh,id_s,t_rd,t_cap
2,1,NaN,1.png,NaN,NaN
4,2,NaN,2.png,NaN,NaN
5,3,23.0,3.png,23.0,160.0
12,4,NaN,4.png,NaN,105.0
13,5,NaN,5.png,NaN,NaN
14,6,NaN,6.png,NaN,NaN
17,7,NaN,7.png,NaN,NaN
19,8,NaN,8.png,16.5,65.0
22,9,NaN,9.png,NaN,NaN
23,10,30.0,10.png,27.0,225.0
